In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
# from sklearn.datasets import fetch_openml
# mnist = fetch_openml('mnist_784', version=1)
# mnist.keys()

In [3]:
# X, y = mnist['data'], mnist['target']

In [4]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=42)

### Getting Started with Pytorch

In [40]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)


In [41]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((X_train, y_train), (X_test, y_test), _) = pickle.load(f, encoding="latin-1")

In [42]:
import torch


X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train, y_train, X_test, y_test = map(
    torch.tensor, (X_train, y_train, X_test, y_test)
)


bs = 64

In [43]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

In [44]:
from torch.utils.data import DataLoader

def get_data(train_ds, test_ds, bs=64):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(test_ds, batch_size=bs * 2),
    )

In [45]:
train_dl, test_dl = get_data(train_ds, test_ds)

In [46]:
import torch.nn.functional as F
loss_func = F.cross_entropy

In [47]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

In [48]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

In [49]:
def preprocess(x, y, p=28):
    return x.view(-1, 1, p, p), y


class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))

train_dl = WrappedDataLoader(train_dl, preprocess)
test_dl = WrappedDataLoader(test_dl, preprocess)

In [50]:
from IPython.core.debugger import set_trace

import numpy as np


def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    epoch_train_loss = []
    epoch_test_loss = []

    for epoch in range(epochs):
        model.train()
        train_losses = []
        train_nums = []
        for xb, yb in train_dl:
            losses, nums = loss_batch(model, loss_func, xb, yb, opt)
            train_losses.append(losses)
            train_nums.append(nums)
        train_loss = np.sum(np.multiply(np.array(train_losses), np.array(train_nums))) / np.sum(train_nums)
        epoch_train_loss.append(train_loss)


        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in test_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        epoch_test_loss.append(val_loss)

        print('epoch #: {}, train_loss: {}, val_loss: {}'.format(epoch, train_loss, val_loss))
        print('train_accuracy: {}, val_accuracy: {}'.format(accuracy(model(X_train.view(-1,1,28,28)), y_train), accuracy(model(X_test.view(-1,1,28,28)), y_test)))
        print('=======================================')

    return = epoch_train_loss, epoch_test_loss


In [51]:
from torch import nn

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [58]:
model = nn.Sequential(
    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    # Lambda(lambda x: x.view(x.size(0), -1)),
    nn.Flatten(),
)

from torch import optim

lr = 0.5  # learning rate
epochs = 15
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

fit(epochs, model, loss_func, opt, train_dl, test_dl)

epoch #: 0, train_loss: 0.9990334366989135, val_loss: 0.6126176641464234
train_accuracy: 0.8108000159263611, val_accuracy: 0.8306000232696533
epoch #: 1, train_loss: 0.4234979261302948, val_loss: 0.3990812261104584
train_accuracy: 0.8687800168991089, val_accuracy: 0.8787000179290771
epoch #: 2, train_loss: 0.3782279883813858, val_loss: 0.37200057525634767
train_accuracy: 0.8808199763298035, val_accuracy: 0.8899000287055969
epoch #: 3, train_loss: 0.34477763493537905, val_loss: 0.2861241530478001
train_accuracy: 0.9217000007629395, val_accuracy: 0.9280999898910522
epoch #: 4, train_loss: 0.3223202053833008, val_loss: 0.3282582637667656
train_accuracy: 0.8959000110626221, val_accuracy: 0.9042999744415283
epoch #: 5, train_loss: 0.29471539058685303, val_loss: 0.2514480815410614
train_accuracy: 0.9236599802970886, val_accuracy: 0.9271000027656555
epoch #: 6, train_loss: 0.2965756346416473, val_loss: 0.24965343329906464
train_accuracy: 0.9161400198936462, val_accuracy: 0.9262999892234802
ep

### CNNs for CIFAR-10

In [59]:
from sklearn.datasets import fetch_openml
cifar_10_small = fetch_openml('CIFAR_10_Small', version=1)
cifar_10_small.keys()

dict_keys(['data', 'target', 'frame', 'feature_names', 'target_names', 'DESCR', 'details', 'categories', 'url'])

In [60]:
X_train, y_train = cifar_10_small['data'], cifar_10_small['target']

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

NameError: ignored

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    # Lambda(lambda x: x.view(x.size(0), -1)),
    nn.Flatten(),
)

from torch import optim

momentum = 0.9
lr = 0.5  # learning rate
epochs = 10
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_losses, test_losses = fit(epochs, model, loss_func, opt, train_dl, test_dl)

In [ ]:
plt.plot(train_losses, range(len(train_losses)), 'b-', legend='train' )
plt.plot(test_losses, range(len(test_losses)), 'g--', legend='test')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Model 1: train_vs_test')

In [ ]:
model2 = nn.Sequential(
    nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(64*16*16, 64),
    nn.ReLU(),
    nn.Linear(64, 19)
)

from torch import optim

momentum = 0.9
lr = 0.5  # learning rate
epochs = 10
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_losses, test_losses = fit(epochs, model, loss_func, opt, train_dl, test_dl)

In [ ]:
plt.plot(train_losses, range(len(train_losses)), 'b-', legend='train' )
plt.plot(test_losses, range(len(test_losses)), 'g--', legend='test')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('Model 2: train_vs_test')